In [1]:
import os
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFDirectoryLoader


In [2]:
from dotenv import load_dotenv
load_dotenv(".env")

True

In [3]:
os.environ["AZURE_OPENAI_API_KEY"] = os.environ.get("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = os.environ.get("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_VERSION"] = os.environ.get("AZURE_OPENAI_API_VERSION")


In [4]:
from langchain.chat_models import AzureChatOpenAI
llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    verbose=False,
    temperature=0.3,
)

/var/folders/pf/r0cmnchs509dgvhzjm4qsn640000gn/T/ipykernel_75900/2845381590.py:2: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(


In [5]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)
prompt = "Provide a detailed company overview for a pitch book presentation."
messages = [
    SystemMessage(content=f"You are an assistant helping with investment banking slides.  {prompt}" )                
]
     

In [6]:
response = llm(
        messages= messages,
        max_tokens=100,
        temperature=0.2,
    )

response.content

/var/folders/pf/r0cmnchs509dgvhzjm4qsn640000gn/T/ipykernel_67793/3769255969.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(


'Certainly! Below is a detailed company overview for a pitch book presentation. This template can be customized based on the specific company being presented:\n\n---\n\n### [Company Name] Overview\n\n#### 1. **Company Snapshot**\n   - **Name:** [Company Name]\n   - **Founded:** [Year]\n   - **Headquarters:** [City, State, Country]\n   - **Industry:** [Industry Sector]\n   - **Ticker Symbol:** [Ticker] (if publicly traded)\n   - **Website'

In [6]:
embeddings = AzureOpenAIEmbeddings(
        azure_deployment=os.environ.get("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT"),
        openai_api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
    )

In [7]:
def create_vector_database(txt_path):
    loader=PyPDFDirectoryLoader(txt_path)
    docs=loader.load()

    documents = RecursiveCharacterTextSplitter(
        chunk_size=1000, separators=["\n","\n\n"], chunk_overlap=200
    ).split_documents(docs)

    embeddings = AzureOpenAIEmbeddings(
        azure_deployment=os.environ.get("AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT"),
        openai_api_version=os.environ.get("AZURE_OPENAI_API_VERSION"),
    )
    db = FAISS.from_documents(
        documents=documents,
        embedding=embeddings
    )
    db.save_local("./faiss-db")

In [14]:
create_vector_database('10k')

In [8]:
from langchain.prompts import PromptTemplate

In [9]:
prompt_template = """
Human: Use the following pieces of context to provide a 
concise answer to the question at the end but use atleast summarize with 
250 words with detailed explantions. If you don't know the answer, 
just say that you don't know, don't try to make up an answer.
<context>
{context}
</context

Question: {question}

Assistant:"""

In [10]:
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [11]:
vectorstore = FAISS.load_local("./faiss-db", embeddings, allow_dangerous_deserialization=True)

In [12]:
question = "Get the company performance"

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [22]:
rag_chain = (
    {"context": vectorstore.as_retriever(search_kwargs={"k": 1}), "question": RunnablePassthrough()}
    | PROMPT
    | llm
    | StrOutputParser()
)

In [23]:
print(rag_chain.invoke(question))

The provided context details the five-year cumulative total shareholder return for Apple Inc., comparing it with the S&P 500 Index and the Dow Jones U.S. Technology Supersector Index. This comparison is based on a dividend-reinvested basis, meaning any dividends paid out by the companies are assumed to be reinvested back into the stock. This method gives a more accurate picture of the total returns an investor would receive.

The graph and data show the performance from September 28, 2018, to September 30, 2023. Here are the key points:

1. **Initial Investment**: The graph assumes an initial investment of $100 in Apple Inc., the S&P 500 Index, and the Dow Jones U.S. Technology Supersector Index as of September 28, 2018.

2. **Apple Inc. Performance**:
   - September 28, 2018: $100
   - September 28, 2019: $98
   - September 26, 2020: $204
   - September 25, 2021: $269
   - September 24, 2022: $277
   - September 30, 2023: $317

   This data indicates that Apple's stock value increased

In [21]:
len(retriever.get_relevant_documents(question)[0].page_content.split(" "))

150